In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = 'cs231n/assignments/assignment1/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# This downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd /content/drive/My\ Drive/$FOLDERNAME/cs231n/datasets/
!bash get_datasets.sh
%cd /content/drive/My\ Drive/$FOLDERNAME

# 图像特征练习
*请完成并提交这份完整的作业工作表（包括输出和任何工作表之外的辅助代码）。更多详情请查看课程网站上的[作业页面](http://vision.stanford.edu/teaching/cs231n/assignments.html)。*

我们可以看到，通过在输入图像的像素上训练线性分类器，我们可以在图像分类任务上获得合理的性能。在这个练习中，我们将证明通过在从原始像素计算的特征上训练线性分类器，我们可以提高分类性能。

你在这个练习中的所有工作都将在这个 notebook 中完成。

In [ ]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt


%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # 设置绘图的默认大小
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# 用于自动重新加载外部模块
# 参见 http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## 加载数据
与之前的练习类似，我们将从磁盘加载 CIFAR-10 数据。

In [ ]:
from cs231n.features import color_histogram_hsv, hog_feature

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000):
    # 加载原始 CIFAR-10 数据
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'

    # 清理变量以防止多次加载数据（这可能导致内存问题）
    try:
       del X_train, y_train
       del X_test, y_test
       print('清除之前加载的数据。')
    except:
       pass

    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

    # 对数据进行子采样
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]

    return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()

## 提取特征
对于每张图像，我们将计算一个定向梯度直方图（HOG）以及一个使用 HSV 色彩空间中色相通道的色彩直方图。我们通过连接
HOG 和色彩直方图特征向量为每张图像形成最终的特征向量。

粗略地说，HOG 应该捕获图像的纹理而忽略色彩信息，而色彩直方图表示输入图像的色彩而忽略纹理。因此，我们期望两者一起使用比单独使用任何一种效果更好。验证这一假设对你的兴趣来说是一个很好的尝试。

`hog_feature` 和 `color_histogram_hsv` 函数都在单张图像上操作，并返回该图像的特征向量。extract_features
函数接收一组图像和一个特征函数列表，并评估
每个特征函数在每张图像上，将结果存储在一个矩阵中，其中
每列是单张图像的所有特征向量的连接。

In [ ]:
from cs231n.features import *

# num_color_bins = 10 # 色彩直方图中的 bin 数量
num_color_bins = 25 # 色彩直方图中的 bin 数量
feature_fns = [hog_feature, lambda img: color_histogram_hsv(img, nbin=num_color_bins)]
X_train_feats = extract_features(X_train, feature_fns, verbose=True)
X_val_feats = extract_features(X_val, feature_fns)
X_test_feats = extract_features(X_test, feature_fns)

# 预处理：减去平均特征
mean_feat = np.mean(X_train_feats, axis=0, keepdims=True)
X_train_feats -= mean_feat
X_val_feats -= mean_feat
X_test_feats -= mean_feat

# 预处理：除以标准差。这确保每个特征
# 大致具有相同的尺度。
std_feat = np.std(X_train_feats, axis=0, keepdims=True)
X_train_feats /= std_feat
X_val_feats /= std_feat
X_test_feats /= std_feat

# 预处理：添加偏置维度
X_train_feats = np.hstack([X_train_feats, np.ones((X_train_feats.shape[0], 1))])
X_val_feats = np.hstack([X_val_feats, np.ones((X_val_feats.shape[0], 1))])
X_test_feats = np.hstack([X_test_feats, np.ones((X_test_feats.shape[0], 1))])

## 在特征上训练 Softmax 分类器
使用作业前面开发的 Softmax 代码，在上面提取的特征之上训练 Softmax 分类器；这应该比直接在原始像素上训练它们获得更好的结果。

In [ ]:
# 使用验证集来调整学习率和正则化强度

from cs231n.classifiers.linear_classifier import Softmax

learning_rates = [1e-7, 1e-6]
regularization_strengths = [5e5, 5e6]

results = {}
best_val = -1
best_softmax = None

################################################################################
# TODO:                                                                        #
# 使用验证集设置学习率和正则化强度。这应该与 Softmax 的验证相同；保存   #
# 最佳训练分类器于 best_softmax。如果你仔细调整模型，                     #
# 你应该能够在验证集上获得超过 0.42 的准确率。                               #
################################################################################


# 打印结果。
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e 训练准确率: %f 验证准确率: %f' % (
                lr, reg, train_accuracy, val_accuracy))

print('最佳验证准确率: %f' % best_val)

In [ ]:
# 评估你训练好的 Softmax 在测试集上：你应该至少得到 0.42
y_test_pred = best_softmax.predict(X_test_feats)
test_accuracy = np.mean(y_test == y_test_pred)
print(test_accuracy)

In [ ]:
# 保存最佳 softmax 模型
best_softmax.save("best_softmax_features.npy")

In [ ]:
# 获得算法如何工作的直觉的一个重要方式是
# 可视化它犯的错误。在这个可视化中，我们显示我们当前系统错误分类的图像示例。第一列
# 显示我们系统标记为"plane"但其真实标签不是"plane"的图像。

examples_per_class = 8
classes = ['飞机', '汽车', '鸟', '猫', '鹿', '狗', '青蛙', '马', '船', '卡车']
for cls, cls_name in enumerate(classes):
    idxs = np.where((y_test != cls) & (y_test_pred == cls))[0]
    idxs = np.random.choice(idxs, examples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt.subplot(examples_per_class, len(classes), i * len(classes) + cls + 1)
        plt.imshow(X_test[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls_name)
plt.show()

### 内联问题 1：
描述你看到的错误分类结果。有道理吗？


$\color{blue}{\textit 你的答案：}$





## 在图像特征上的神经网络
在这个作业的前面部分，我们看到在原始像素上训练两层神经网络比在原始像素上的线性分类器获得更好的分类性能。在这个 notebook 中，我们看到在图像特征上的线性分类器比在原始像素上的线性分类器表现更好。

为了完整起见，我们也应该尝试在图像特征上训练神经网络。这种方法应该优于所有以前的方法：你应该很容易在测试集上获得超过 55% 的分类准确率；我们的最佳模型获得大约 60% 的分类准确率。

In [ ]:
# 预处理：移除偏置维度
# 确保只运行一次此单元格
print(X_train_feats.shape)
X_train_feats = X_train_feats[:, :-1]
X_val_feats = X_val_feats[:, :-1]
X_test_feats = X_test_feats[:, :-1]

print(X_train_feats.shape)

In [ ]:
from cs231n.classifiers.fc_net import TwoLayerNet
from cs231n.solver import Solver

input_dim = X_train_feats.shape[1]
hidden_dim = 500
num_classes = 10

data = {
    'X_train': X_train_feats,
    'y_train': y_train,
    'X_val': X_val_feats,
    'y_val': y_val,
    'X_test': X_test_feats,
    'y_test': y_test,
}

net = TwoLayerNet(input_dim, hidden_dim, num_classes)
best_net = None

################################################################################
# TODO: 在图像特征上训练两层神经网络。你可能想要像之前部分一样进行各种参数的交叉验证。将你最好的模型存储在 best_net 变量中。 #
################################################################################

In [ ]:
# 运行你最好的神经网络分类器在测试集上。你应该能够
# 得到超过 58% 的准确率。通过仔细调整，也有可能会得到 >60% 的准确率。

y_test_pred = np.argmax(best_net.loss(data['X_test']), axis=1)
test_acc = (y_test_pred == data['y_test']).mean()
print(test_acc)

In [ ]:
# 保存最佳模型
best_net.save("best_two_layer_net_features.npy")